In [1]:
from fastai.text.all import *

# Fastai Text Classification - Pretraining a LM and Classifier

### Get the IMDB dataset

In [2]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_clas')]

### Load the data (only LM)

In [3]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)

### Show some sample data

In [4]:
dls_lm.show_batch()

,text,text_
0,xxbos xxup the xxup good : xxmaj the acting were great especially xxmaj terrence xxmaj howard and xxmaj thandie xxmaj newton . xxmaj terrence xxmaj howard should have been nominated for xxmaj best xxmaj supporting xxmaj actor instead of xxmaj matt xxmaj dillon . \n\n xxup the xxup bad : xxmaj i 'm a visible minority ( non - white ) and i have experienced some form of racism in my life,xxup the xxup good : xxmaj the acting were great especially xxmaj terrence xxmaj howard and xxmaj thandie xxmaj newton . xxmaj terrence xxmaj howard should have been nominated for xxmaj best xxmaj supporting xxmaj actor instead of xxmaj matt xxmaj dillon . \n\n xxup the xxup bad : xxmaj i 'm a visible minority ( non - white ) and i have experienced some form of racism in my life .
1,with honesty . xxmaj it was so nice to see a film where the story line centered on young professional african - americans . xxmaj this is virtually an untapped market . xxmaj love xxmaj jones was visually captivating as well . xxmaj the chemistry between xxmaj lorenz xxmaj tate and xxmaj nia xxmaj long will bring memories of past and present love . xxmaj the feeling of the film is jazz,honesty . xxmaj it was so nice to see a film where the story line centered on young professional african - americans . xxmaj this is virtually an untapped market . xxmaj love xxmaj jones was visually captivating as well . xxmaj the chemistry between xxmaj lorenz xxmaj tate and xxmaj nia xxmaj long will bring memories of past and present love . xxmaj the feeling of the film is jazz and
2,"admit that it was a good film . xxmaj the quality was a little worse for the wear , but it was a well acted and realistic drama involving low level xxmaj new xxmaj jersey gangsters . xxmaj pesci once again though , steels the show ! xxbos xxmaj what happened to the sweet , precocious little girl we all grew up with ? xxmaj when the series was canceled , i","that it was a good film . xxmaj the quality was a little worse for the wear , but it was a well acted and realistic drama involving low level xxmaj new xxmaj jersey gangsters . xxmaj pesci once again though , steels the show ! xxbos xxmaj what happened to the sweet , precocious little girl we all grew up with ? xxmaj when the series was canceled , i felt"
3,"xxmaj of course the snob could n't even do that right . i have enough problems with xxmaj gwyneth xxmaj paltrow as xxmaj gwyneth xxmaj paltrow , and her "" emma "" did not exactly change my opinion . \n\n xxmaj well , it 's easy to see that i did not care for this one . xxmaj i 'm sure it 's a lovely book and all , but some books","of course the snob could n't even do that right . i have enough problems with xxmaj gwyneth xxmaj paltrow as xxmaj gwyneth xxmaj paltrow , and her "" emma "" did not exactly change my opinion . \n\n xxmaj well , it 's easy to see that i did not care for this one . xxmaj i 'm sure it 's a lovely book and all , but some books are"
4,his badge he searches for the last vampire slayer ( no not xxmaj buffy ! ) to take back the streets and vanquish the bloodsuckers back to the grave . \n\n ' vampire xxmaj assassins ' is a horrible film . xxmaj it rips off so many other films ( can you say xxmaj blade ? ) that it never even attempts to establish it 's own identity . xxmaj the script,badge he searches for the last vampire slayer ( no not xxmaj buffy ! ) to take back the streets and vanquish the bloodsuckers back to the grave . \n\n ' vampire xxmaj assassins ' is a horrible film . xxmaj it rips off so many other films ( can you say xxmaj blade ? ) that it never even attempts to establish it 's own identity . xxmaj the script is
5,"xxunk xxmaj sutherland ) who is looking for the worlds most notorious terrorist and add a xxmaj ben xxmaj kingsley and you have "" the xxmaj assignment "" . xxmaj sutherland is a witness of the most notorious terrorist xxmaj carlos actions in a cafe on a lovely day where he is so profoundly rocked at this mans evil that his sole reason to

### Build a Language Model Learner
- Use `AWD_LSTM` as pretrained model
- Use `accuracy` and `perplexity` as metrics
- Use `to_fp16()` for faster computations

In [5]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

### Fit one cycle

In [6]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.399793,4.105767,0.286672,60.689289,30:26


### Save the trained model

In [7]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

### Load the trained model

In [8]:
learn = learn.load('1epoch')

### Unfreeze the model and train the model

In [9]:
learn.unfreeze()
learn.fit_one_cycle(3, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.066998,3.891075,0.306145,48.963490,33:09
1,3.972945,3.805165,0.315330,44.932667,33:27
2,3.904613,3.759892,0.320839,42.943810,33:09


### Save the encoder (except the last layers)

In [10]:
learn.save_encoder('finetuned')

### Try our very own pretrained LM for generating text reviews

In [11]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [12]:
print("\n".join(preds))

i liked this movie because i did n't expect a lot from Dennis Quaid ( and i already liked the " pirates " in it ) . The acting was n't very good , the actors were great , the actors were
i liked this movie because i loved the performances of Julia Roberts , Kevin Costner and Martin Sheen . The story is full of violence , exotic locations and some of the worst action scenes in the film


## Finetuning the classifier
### Load the pretrained model with the finetuned LM vocabulary

In [13]:
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', text_vocab=dls_lm.vocab)

### Build the classifier

In [14]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

### Load the saved encoder to the classifier 

In [15]:
learn = learn.load_encoder('finetuned')

### Fit for one cycle

In [16]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.259396,0.205606,0.917880,02:15


## Finetuning the Classifier model
- Freeze the last two layers and train

In [17]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.228839,0.180698,0.932040,02:33


- Freeze the last three layers and train

In [18]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.193656,0.163980,0.938360,03:26


- Unfreeze all layers and train

In [19]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.147810,0.160574,0.940040,04:13
1,0.127101,0.158921,0.941200,04:13


## Is it any better than the previous version?